In [1]:
import spacy, os
nlp = spacy.load('en')

In [2]:
# switch for debug
flag_print = True

# switch to clear existing data
flag_clear = True

#threshold value for determining section
threshold = 0.5

In [14]:
# to get extract sections from the resume -- add or remove from  'similar_to' accordingly
similar_to = {
    'edu' : ['education', 'study', 'academics'],
    'exp' : ['job', 'internship', 'training', 'research', 'carrer', 'profession', 
             'project', 'responsibility', 'description'],
    'skill' : ['skill', 'languages', 'technology', 'frameworks', 'tools'],
    'extra' : ['introduction', 'intro', 'achievement', 'hobby', 'links', 'additional', 'personal']
}

list_of_sections = similar_to.keys()

# to bring similar_words to their normal forms
for section in list_of_sections:
    new_list = []
    
    for word in similar_to[section]:
        docx = nlp(unicode(word))
        new_list.append(docx[0].lemma_)
        
    if flag_print:
        print section, new_list
        
    similar_to[section] = new_list

skill [u'skill', u'language', u'technology', u'framework', u'tool']
extra [u'introduction', u'intro', u'achievement', u'hobby', u'link', u'additional', u'personal']
exp [u'job', u'internship', u'training', u'research', u'carrer', u'profession', u'project', u'responsibility', u'description']
edu [u'education', u'study', u'academic']


In [4]:
# function to return the words in a uniform 
def modify(word):
    try:
        symbols = '''~'`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_word = ''
        
        for char in word:
            if (char not in symbols):
                mod_word += char.lower()

        docx = nlp(unicode(mod_word))

        if (len(mod_word) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return None # to handle the odd case of characters like 'x02', etc.
    
if flag_print:
    test_words = ['Hello!!', '.,<>', 'India', 'of', '..freedoM..', 'e-mail']
    
    for word in test_words:
        print word, '--returned-->', modify(word)

Hello!! --returned--> hello
.,<> --returned--> None
India --returned--> india
of --returned--> None
..freedoM.. --returned--> freedom
e-mail --returned--> email


In [24]:
# utility function to skip line when no alphabet present
def is_bad(line):
    for c in line:
        if (c.isalpha()):
            return False
    return True
      
if flag_print:
    test_words = ['.', '<.>', 'Speak', 'out', '"Eric"', 'freemail...']
    
    for word in test_words:
        print word, '--returned-->', is_bad(word)  

. --returned--> True
<.> --returned--> True
Speak --returned--> False
out --returned--> False
"Eric" --returned--> False
freemail... --returned--> False


In [33]:
if flag_clear:
    for file_name in os.listdir(os.getcwd()+'/prc_data'):
        os.remove(os.getcwd()+'/prc_data/'+file_name)

for file_name in os.listdir(os.getcwd()+'/raw_data'):
    if flag_print:
        print '\n'
        print '*'*25
        print file_name
        print '*'*25
        
    main_file_handler = open('raw_data/'+file_name, 'r')    
    previous_section  = 'extra'
    
    for line in main_file_handler:
        # skip line if empty
        if (len(line.strip()) == 0 or is_bad(line)):
            continue
                
        # processing next line
        list_of_words_in_line = line.split()
        list_of_imp_words_in_line  = []
        
        for i in range(len(list_of_words_in_line)):
            modified_word = modify(list_of_words_in_line[i])
            
            if (modified_word):
                list_of_imp_words_in_line.append(modified_word)

        if (True):
            curr_line = ' '.join(list_of_imp_words_in_line)
            doc = nlp(unicode(curr_line))
            section_value = {}
            
            # initializing section values to zero
            for section in list_of_sections:
                section_value[section] = 0.0
            section_value[None] = 0.0
            
            # updating section values    
            for token in doc:
                for section in list_of_sections:
                    for word in similar_to[section]:
                        word_token = doc.vocab[unicode(word)]
                        section_value[section] = max(section_value[section], float(word_token.similarity(token)))

            # determining the next section based on section values and threshold
            most_likely_section = None
            for section in list_of_sections:
                #print '>>', section, section_value[section]
                if (section_value[most_likely_section] < section_value[section] and section_value[section] > threshold):
                    most_likely_section = section
            
            # updating the section
            if (previous_section != most_likely_section and most_likely_section is not None):
                previous_section = most_likely_section
                

        # writing data to the separate files
        temp_file_handler = open('prc_data/'+file_name[:-4]+'_'+previous_section+'.txt', 'a')
        try:
            docx = nlp(unicode(line))
        except:
            continue  # to handle the odd case of characters like 'x02', etc.
        mod_line = ''
        for token in docx:
            if (not token.is_stop):
                mod_line += token.lemma_ + ' '
        temp_file_handler.write(mod_line)
        temp_file_handler.close()
        
        if flag_print:
            print previous_section, '\t**', mod_line.strip()
        
    main_file_handler.close()
        



*************************
cv2.txt
*************************
extra 	** puneet singh
extra 	** new delhi , delhi
skill 	** work professional industry professional work drive environment utilize apply knowledge , skill ability beneficial mutual growth .
extra 	** personal trait
exp 	** professional qualification
exp 	** work experience
skill 	** software developer
skill 	** software - noida , uttar pradesh
skill 	** fresh
skill 	** skill
exp 	** project
edu 	** team size : 1
exp 	** role : developer
skill 	** technology require : .net end , sql server end
exp 	** project manage medicine inventory system medical store system .
exp 	** team size : 2
exp 	** role : developer
skill 	** technology require : c # end , sql server end
exp 	** project mainly allahabad medicos , allahabad .
skill 	** team size : 2
exp 	** role : developer
skill 	** technology require : .net end , sql server end
extra 	** additional information


*************************
cv52.txt
*************************
exp 	**

skill 	** kharagpur , wb
exp 	** dual degree computer science & engineering ;
exp 	** cgpa:9.24
edu 	** lucknow public school
edu 	** lucknow ,
edu 	** marks:97.2%
edu 	** sacred heart school
edu 	** lucknow ,
edu 	** india secondary school examination , aisse ;
edu 	** marks:90.4%
exp 	** internship
exp 	** ibm research lab
exp 	** new delhi , india
edu 	** community discovery social networks pitman - yor process
exp 	** ibm research lab
exp 	** new delhi , india
exp 	** discover topic behaviour unstructured datum
exp 	** award
exp 	** best poster
extra 	** poster presentation competition
exp 	** project
edu 	** topic dependent community discovery social networks pitman - yor process
exp 	** guide - prof . pabitra mitra , department computer science & engineering ( july 2011 - present )
skill 	** propose model capture power - law property graph communication exchange
skill 	** discover event spatio - temporal data mobile users
exp 	** guide - prof . sudeshna sarkar , dept . computer s

edu 	** cholesky factorization parallel architecture
edu 	** july 2010 - november 2010
skill 	** cs738 :   advanced compiler optimization
skill 	** dept .   cse , iitk
skill 	** advisor :
skill 	** prof .   sanjeev k. aggarwal
exp 	** description :
exp 	** project aim exploit innate parallelism available algorithm cholesky fac-
extra 	** torization symmetric positive - definite matrix achieve high speedup .   block algorithm available
skill 	** lapack implement level-3 basic linear algebra operation square block pack ( sbp )
extra 	** format & optimize compiler optimization technique .   performance compare multiple hard-
skill 	** ware cuda ( gpgpu ) & intel tbb ( 2,4,16,24 core ) .   result parallelize block algorithm
skill 	** new digital watermarking scheme 3d triangular mesh model
skill 	** december 2009 - 2010
skill 	** cs698e : digital watermarking & steganography
skill 	** dept .   cse , iitk
skill 	** advisor :
skill 	** prof .   phalguni gupta
exp 	** description :
exp 	** pr

extra 	** place :    chennai
extra 	** date : 																				                                      ( signature )
extra 	** r.abhishek
